In [27]:
from itertools import product
from random import shuffle

from language_models.character_frequency_kld_language_model import CharacterFrequencyKLDLanguageModel

In [28]:
# Imports de Enigma
from enigma.enigma import Enigma
from enigma.plugboard import Plugboard
from enigma.reflectors.reflector import Reflector
from enigma.reflectors.reflector_b import ReflectorB
from enigma.reflectors.reflector_b_thin import ReflectorBThin
from enigma.reflectors.reflector_c import ReflectorC
from enigma.rotors.rotor import Rotor
from enigma.rotors.rotor_I import RotorI
from enigma.rotors.rotor_II import RotorII
from enigma.rotors.rotor_III import RotorIII
from enigma.rotors.rotor_IV import RotorIV

In [94]:
ENGLISH_ALPHABET_CHARACTERS = "abcdefghijklmnopqrstuvwxyz"
ENGLISH_ALPHABET_LEN = len(ENGLISH_ALPHABET_CHARACTERS)
# TODO: Is this a parameter?
HARDCODED_PERIOD = 26

with open('books/Frankenstein.txt', 'r') as book:
    frankestein_book = text = [c for c in "".join(book.readlines()).lower() if c in ENGLISH_ALPHABET_CHARACTERS]

PLUGBOARD_TUPLES = [('A', 'F'), ('G', 'H'), ('Y', 'S'), ('M', 'T')]

In [95]:
# Me defino una enigma
enigma = Enigma(reflector=Reflector(),
                        plugboard=Plugboard(PLUGBOARD_TUPLES),
                        rotors=[RotorIII(offset=2),
                                RotorII(offset=3),
                                RotorI(offset=5)])

In [96]:

# Modelo de lenguaje basado en la frecuencia de caracteres en un libro en ingles
with open('books/Alices_Adventures_in_Wonderland.txt', 'r') as book:
    freq_kld_model = CharacterFrequencyKLDLanguageModel(book.read())   

with open('books/Frankenstein.txt', 'r') as book:
    all_book = "".join(book.readlines()).lower()
    frankestein_book = "".join([c for c in all_book if c in ENGLISH_ALPHABET_CHARACTERS])

In [99]:
# Me genero todas las posibles combinaciones de los offsets de los rotores
def crack(language_model, encrypted_message):
    offset_rotor_combinations = list(product(range(ENGLISH_ALPHABET_LEN), repeat = 3))
    print(len(offset_rotor_combinations))
    kb_divergences = {}
    i=0
    for combination in combinations_rotores:
        enigma_with_given_offset = Enigma(Reflector(),
                                          plugboard=Plugboard(PLUGBOARD_TUPLES),
                                          rotors=[
                                              RotorIII(offset=combination[0]),
                                              RotorII(offset=combination[1]),
                                              RotorI(offset=combination[2])])
        decrypted_message = enigma_with_given_offset.decrypt(encrypted_message)
        kb_divergences[combination] = freq_kld_model.fitness(decrypted_message)
        if(i % 10000 == 0): print(i, end = ' ')
        i+=1
    sorted_divergences = sorted(kb_divergences.items(), key=lambda x: x[1])
    return sorted_divergences[:50]

In [100]:
%%time
type(frankestein_book)
crack(freq_kld_model, frankestein_book[100:200])

17576
0 10000 20000 30000 

KeyboardInterrupt: 